In [1]:
import pandas as pd
import re
import numpy as np
from collections import defaultdict
import networkx as nx
import netwulf as nw
import ast

# Import data and functions

In [6]:
#read dataset of tweets since election and list of our actors
klima_df=pd.read_csv('climate_tweets_since_election.zip')
df2=pd.read_excel('actor_list.xlsx')

In [7]:
#remove duplicates
klima_df=klima_df.drop_duplicates(subset='tweet_id')
klima_df = klima_df.loc[df.tweet_created_at > '05-06-2020']

In [8]:
#define extract functions
def extract_list(string):
    """Helper function to extract dict from string if string exists 
    else the function returns an empty dict (Thanks, Tweepy...)"""
    try:
        out=ast.literal_eval(string)
    except:
        out=list()
    return out


def extract_dict(string):
    """Helper function to extract dict from string if string exists 
    else the function returns an empty dict (Thanks, Tweepy...)"""
    try:
        out=ast.literal_eval(string)
    except:
        out=dict()
    return out

def extract_from_entities(tweet,ent_key,tag_key):
    """Helper function to extract information from tweet_entities.
    tweet_entities is a dict-of-dicts containing all information on 
    twitter entities from a given tweet.
    ent_key: key used to access the dictionary of interest e.g. "hastags"
    tag_key: key used to access value of interest e.g. "text"
    Why, Tweepy? WHY?!"""
    try:
        out=[tag[tag_key] for tag in tweet[ent_key] if tweet[ent_key]!=ent_key]
    except:
        out=list()
    return out

In [9]:
#Extract tweet entities dict into seperate columns

klima_df["tweet_retweeted_status"]=klima_df["tweet_retweeted_status"].apply(lambda x:extract_dict(x))
klima_df["tweet_quoted_status"]=klima_df["tweet_quoted_status"].apply(lambda x:extract_dict(x))
klima_df["tweet_entities"]=klima_df["tweet_entities"].apply(lambda x:extract_dict(x))

klima_df["tweet_hashtags"]=klima_df.tweet_entities.apply(lambda tweet: extract_from_entities(tweet,"hashtags","text"))
klima_df["tweet_mentions"]=klima_df.tweet_entities.apply(lambda tweet: extract_from_entities(tweet,"user_mentions","screen_name"))

# Create dict

In [55]:
#Create dict of accounts and who they mention
mention_dict = defaultdict(list)
for idx,row in klima_df.iterrows():
    mention_dict[row['user_screen_name']].append(row['tweet_mentions'])

In [57]:
#keep only alphanumerical characters
mention_dict2 = {key: value for key, value in mention_dict.items() if key.isalpha()}

#flatten the lists in dict
for key in mention_dict2.keys():
    mention_dict2[key]=[item for sublist in mention_dict2[key] for item in sublist]

# Network

In [61]:
#create network from dict
mention_network=nx.from_dict_of_lists(mention_dict2)

In [ ]:
#create list of nodes that are not in our actor list
not_rel=[]
for i in list(mention_network.nodes()):
    if i not in df2['actors'].to_list():
        if i not in list(klima_df.user_screen_name.unique()):
            if i not in df2['actors'].str.lower():
                not_rel.append(i)

In [ ]:
#remove 
for i in not_rel:
    mention_network.remove_node(i)